# Estadística para Data Science: Cuaderno clase 7

In [2]:
# Importando librerías

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy.stats as stats
import seaborn as sns


# Retomando la última clase, Inferencia sobre una proporción: Intervalo de confianza Y Teorema Central del Límite




## Recordemos: 

* Inferencia: Usar datos y razonamiento sobre la muestra para decir cosas sobre la población.
* Población: El conjunto completo de sujetos de interés
* Parámetro: Una métrica de la población.
* Muestra: Un subconjunto de la población.
* Estadístico de la muestra: una métrica de la muestra
* Estimador:  Un estadístico usado para estimar un parámetro desconocido de la población.
* Distribución de Muestreo de un estadístico: La distribución de frecuencias de un estadístico de la muestra, sobre *muchas* muestras del mismo tamaño, tomadas aleatoriamente desde una población.
* Teorema Central del Límite:  La tendencia de una distribución de muestreo a acercarse a una distribución normal en la medida que el tamaño de la muestra $n$ aumenta. 
* Error estándar: La desviación estándar de un estadístico de la muestra en una distribución de muestreo.

### Construyendo un intervalo de confianza del 95%

Nuestra proporción de la muestra $\hat{p}$ es el valor más plausible de la proporción de la población, por lo que tiene sentido construir un intervalo de confianza en torno a este estimador puntual. El error estándar provee una guía para cuán grande debiésemos hacer el intervalo de confianza.
El error estándar representa la desviación estándar del estimador puntual y cuando se satisfacen las condiciones del Teorema del Límite Central, el estimador puntual se aproxima a una distribución normal.
En una distribución normal, 95% de los datos están dentro de 1.96 desviaciones estándar del promedio.
Usando este principio, podemos construir un intervalo de confianza que se extienda en 1.96 veces el error estándar respecto de la proporción de la muestra para estar **95% seguros** de que el intervalo captura la proporción de la población:

$$point estimate \pm   1.96 \times  SE $$

$$\hat{p} \pm  1.96  \sqrt{\frac{p(1 -p)}{n}} $$

En general en un promedio: 
$$SE =  \sqrt{\frac{\sigma}{n}} $$



Pero, ¿qué significa “95% seguros”? Supongamos que tomamos muchas muestras y construimos un intervalo de confianza de 95% en cada una. Entonces, alrededor del 95% de esos intervalos contendría el parámetro, $p$.

Recordando la clase anterior y la tarea, veamos como sería tomar muchas muestras y construir intervalos de confianza al respecto:


## Datos Mineduc: rendimiento educacion básica y media 2021 en el Bío Bío.
Leamos el siguiente dataset

In [3]:
df_est = pd.read_csv("https://raw.githubusercontent.com/vmlandae/datasets_eds/main/clase5/rendimiento_2021_8va.csv")

Este dataset es un subconjunto de datos a nivel nacional del año 2021 facilitados por el mineduc en su [página de datos abiertos](https://datosabiertos.mineduc.cl/rendimiento-por-estudiante-2/), donde reportan el rendimiento por alumno de enseñanza básica y media. Este dataset fue filtrado y limpiado, para solo contener los estudiantes que se encontraban matriculados en instituciones de la Región del Bío Bío. La [documentación](https://github.com/vmlandae/datasets_eds/blob/main/clase5/ER%20Rendimiento%20por%20alumno%2C%20bases%20Web.pdf) nos permite revisar que es cada variable, pues no todas son tan intuitivas. 

In [4]:
df_est.columns.values

array(['Unnamed: 0', 'AGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD',
       'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD',
       'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2',
       'RURAL_RBD', 'ESTADO_ESTAB', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO',
       'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN',
       'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU',
       'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL',
       'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN'], dtype=object)

In [5]:
# SIT_FIN_R es la situación a final de año de un estudiante,
# donde P: promovido, R: reprobado, Y: Retirado, T: trasladado (se cambió de colegio) y 
# algunos datos en blanco sin información. Vamos a filtrar para dejar solo P,R,Y en el dataset
# porque así estamos contando a cada alumno solo una vez (los que se cambian tienen más de un registro)

df =  df_est.loc[df_est['SIT_FIN_R'].isin(['P','R','Y'])].reset_index()

In [6]:
print(df_est['MRUN'].nunique())
print(df_est.shape)      
print(df['MRUN'].nunique())
print(df.shape)

274781
(280617, 38)
273894
(273894, 39)


Estos datos son censales, por tanto si quisiéramos saber, por ejemplo, la proporción de alumnos de básica y media en la región del Bío Bío el año 2021 que asistieron a un establecimiento municipal, por ejemplo, me basta con calcularlo usando la variable `COD_DEPE2`, que cuando es 1, indica un colegio de dependencia municipal. Entonces

In [7]:
prop_municipal = round( (df['COD_DEPE2']==1).mean(),4)


No vamos a ver cual es el valor real todavía, sino que construiremos distribuciones de muestreo de la proporción haciendo muchos muestreos aleatorios simples (sin reemplazo) para $n=30$, $n=100$, $n=1000$ y $n=5000$, calculando la proporción estimada para cada muestreo y construyendo las tres distribuciones de muestreo de la proporción estimada. Luego, de estas distribuciones, vamos a ver como son las proporciones que estoy estimando con distintas muestras, y como finalmente se comparan con la proporción real.

In [8]:
# creamos variables para almacenar las proporciones estimadas de cada muestreo

municipal30 = []
municipal100 = []
municipal1000 = []
municipal5000 = []


for i in range(1000):
    municipal30.append((df['COD_DEPE2']==1).sample(30).mean())
    municipal100.append((df['COD_DEPE2']==1).sample(100).mean())
    municipal1000.append((df['COD_DEPE2']==1).sample(1000).mean())
    municipal5000.append((df['COD_DEPE2']==1).sample(5000).mean())

## Actividad

1. Calculemos el estimador de la proporción para un muestreo de 500 personas
2. Calculemos el error estándar estimado
3. Calculemos el intervalo de confianza al 90 y al 95% para cada muestreo.
4. para cada $n$, hagamos ahora otros 100 muestreos y calculemos esos intervalos de confianza
5. Calculemos el parámetro de la población, y veamos cuántos de esos 100 intervalos de confianza contienen al parámetro para cada caso.

In [10]:
## escribamos el código juntos


df_est_500 = df.sample(500) 

df_est['COD_DEPE2'].value_counts()


2    137646
1    106318
3     16568
5     15415
4      4670
Name: COD_DEPE2, dtype: int64

In [14]:
df_est_500['COD_DEPE2']==1

13412      True
101740     True
6992       True
115205    False
188059    False
          ...  
46932     False
174588     True
239732    False
29556      True
157785     True
Name: COD_DEPE2, Length: 500, dtype: bool

In [17]:
p_estimado_500 = (df_est_500['COD_DEPE2']==1).mean()

print(p_estimado_500)

0.392


In [18]:
# chequear condiciones para que se cumpla el Teorema Central del Límite, np y n(1-p) deben ser mayores que 10

n=500
print(n*p_estimado_500>10)
print(n*(1-p_estimado_500)>10)


196.0
304.0


In [22]:
# calculando error estándar:


SE_estimado = (p_estimado_500*(1-p_estimado_500)/n)**0.5
print(SE_estimado)

0.021832819332372078


0.042792325891449275

In [21]:
# calcular IC al 95%

IC_inferior = p_estimado_500 - 1.96 * SE_estimado
IC_superior = p_estimado_500 + 1.96 * SE_estimado

# mi IC es:

print('el intervalo de confianza al 95% es de: (',IC_inferior,', ',IC_superior,') ')

el intervalo de confianza al 95% es de: ( 0.34920767410855075 ,  0.4347923258914493 ) 


In [25]:
# calculemos los IC de la proporción para 100 muestreos con n=500

IC_sup_total = []
IC_inf_total = []

for i in range(100):
    x = df['COD_DEPE2'].sample(n)  # n=500
    p_est = (x==1).mean()
    SE_est = (p_est*(1-p_est)/n)**0.5
    IC_est = [ ]
    IC_inf = p_est - 1.96 * SE_est
    IC_sup = p_est + 1.96 * SE_est
    IC_sup_total.append(IC_sup)
    IC_inf_total.append(IC_inf)

    
    

In [28]:
print(IC_sup_total)
print(IC_inf_total)

[0.4388683858898373, 0.44904545556501874, 0.402073855064636, 0.4184577804035962, 0.465290361643211, 0.45514280124424, 0.3959168809526663, 0.4327531669002426, 0.4245889554133463, 0.4449768606019565, 0.4347923258914493, 0.400022326865608, 0.3546815930464873, 0.40617453654517144, 0.4245889554133463, 0.44904545556501874, 0.4347923258914493, 0.4205022734827209, 0.4368307315464025, 0.47339530297163507, 0.402073855064636, 0.391808228855095, 0.4245889554133463, 0.47137015438293944, 0.42867258057347835, 0.38975269269400475, 0.40617453654517144, 0.4510786419841666, 0.4123196635147304, 0.400022326865608, 0.42254599769661066, 0.402073855064636, 0.402073855064636, 0.391808228855095, 0.44904545556501874, 0.39797000315463416, 0.4915892719278494, 0.47137015438293944, 0.4388683858898373, 0.42867258057347835, 0.38358120483102914, 0.40617453654517144, 0.4123196635147304, 0.4388683858898373, 0.4102720702497524, 0.430713252500834, 0.4245889554133463, 0.4327531669002426, 0.45514280124424, 0.4061745365451714

In [34]:
df_intervalos = pd.DataFrame( {'IC_inf':IC_inf_total, 'IC_sup':IC_sup_total})
print(df_intervalos)
print(prop_municipal)

      IC_inf    IC_sup
0   0.353132  0.438868
1   0.362955  0.449045
2   0.317926  0.402074
3   0.333542  0.418458
4   0.378710  0.465290
..       ...       ...
95  0.345287  0.430713
96  0.366889  0.453111
97  0.368857  0.455143
98  0.329634  0.414366
99  0.315978  0.400022

[100 rows x 2 columns]
0.3773


False    94
True      6
Name: IC_inf, dtype: int64

In [39]:
df_intervalos['contiene_valor_real'] = ((df_intervalos['IC_inf']<=prop_municipal) & (df_intervalos['IC_sup']>=prop_municipal))
print(df_intervalos)

      IC_inf    IC_sup  contiene_valor_real
0   0.353132  0.438868                 True
1   0.362955  0.449045                 True
2   0.317926  0.402074                 True
3   0.333542  0.418458                 True
4   0.378710  0.465290                False
..       ...       ...                  ...
95  0.345287  0.430713                 True
96  0.366889  0.453111                 True
97  0.368857  0.455143                 True
98  0.329634  0.414366                 True
99  0.315978  0.400022                 True

[100 rows x 3 columns]


In [41]:
df_intervalos['contiene_valor_real'].value_counts()

True     93
False     7
Name: contiene_valor_real, dtype: int64

## Otro ejemplo

En Nueva York en Octubre del 2014, un doctor que venía de estar tratando ébola en Africa llega al hospital con fiebre y se le diagnostica con ébola, poniéndolo a él y a los contactos estrechos en cuarentena. A los días se realiza una encuesta que encuentra que el 82% de la población de la ciudad está a favor de una cuarentena obligatoria de 21 días para los contactos estrechos. Respondieron la encuesta 1,042 adultos con residencia en la ciudad entre el 26 y 28 de octubre del 2014.

1. ¿Cuál es el estimador puntual y es razonable usar una distribución normal en este modelo?
2. Estimemos el error estándar y construyamos un intervalo de confianza al 95%.
3. ¿Qué significa el 95% de confianza en este contexto?
4. Piensas que el intervalo de confianza todavía sería válido para las opiniones de los ciudadanos de NY al día de hoy?


In [42]:
 (0.82*(1-0.82)/1042)**0.5

0.011901708775845125

In [43]:
 1.96*0.0119

0.023324

## Interpretando intervalos de confianza

Tenemos una confianza del 95% que la proporción de adultos en NY en octubre del 2014 que apoyan una cuarentena para contactos estrechos con un paciente con Ebola estaba entre 0.796 y 0.844.

### Ojo

* El IC es una afirmación sobre el parámetro de la población.
* No es una **probabilidad**

Este último error es muy común. Sería útil si pudiéramos pensar en esto como una probabilidad, pero el IC solo cuantfica cuan plausible es que el parámetro de la población esté en el intervalo dado. Un IC no nos dice nada sobre observaciones individuales ni estimadores puntuales. Recuerda que todo esto solo aplica respecto a errores de muestreo, no a sesgo. Si tomamos una muestra sesgada, esto nos llevará a sistemáticamente sobre o sub estimar el parámetro de la población. Necesitamos mejorar el proceso de recolección de datos para protegernos del sesgo en los ejemplos considerados.
 

In [45]:
import scipy.stats as st
st.norm.ppf(.96)

1.7506860712521692

## Actividad en grupos:

## Esperando en la sala de emergencias 
La administradora de un hospital, esperando mejorar los tiempos de espera, decide estimar el tiempo de espera promedio en la sala de emergencias de su hospital. Ella recolecta una muestra aleatoria simple de 64 pacientes y determina el tiempo (en minutos) entre que se registraron en la sala de emergencias hasta que fueron atendidos por un doctor. Un intervalo de confianza de 95% basado en esta muestra es (128 minutos, 147 minutos), lo que está basado en un modelo normal para el promedio. Determina cuál de los siguientes enunciados son verdaderos o falsos y explica tu razonamiento.

1. Tenemos una confianza de 95% en que el tiempo de espera promedio de estos 64 pacientes de la sala de emergencia está entre 128 y 147 minutos.
2. Tenemos una confianza de 95% en que el tiempo de espera promedio de todos los pacientes de la sala de emergencia de este hospital está entre 128 y 147 minutos.
3.  95% de las muestras aleatorias tienen un promedio de la muestra entre 128 y 147 minutos.
4.  Un intervalo de confianza de 99% sería más estrecho que el intervalo de 95% ya que necesitamos estar más seguros de nuestro estimador.
5.  El margen de error es 9.5 y el promedio de la muestra es de 137.5.
6.  Para disminuir a la mitad el margen de error de un intervalo de confianza de 95% respecto del actual, necesitaríamos duplicar el tamaño de la muestra. (Pista: el margen de error para el promedio escala de la misma manera con el tamaño de la muestra como el margen de error para una proporción).

## Sesgos:
1. La profesión más peligrosa. En una tabulación en 1835 de las edades y profesiones de las personas fallecidas, se encontró que la profesión con el promedio más bajo de edad de muerte era "estudiante". ¿Por qué ser estudiante pareciera ser tan peligroso?
2. Edad y líneas de la mano. Un estudio de 100 personas recientemente fallecidas encontró una correlación positiva fuerte entre la edad de muerte y la longitud de la línea más larga de la mano. ¿Provee esto apoyo a la posición de que una línea larga de la palma predice una vida larga?
3. La ilusión del clínico. Cuando se le pide juzgar la severidad de un síndrome entre sus pacientes, los psiquiatras clínicos tienden a caracterizar los síndromes como mucho más peligrosos y de largo plazo, en promedio, de lo que son estimados por encuestas a los pacientes que tienen el síndrome.
4. Tus amigos son (probablemente) más populares que tú. Los sociólogos han conducido encuestas en que seleccionan personas aleatorias y les piden una lista de las personas que conocen; luego contactan a una muestra de los amigos y repiten la encuesta. Las personas muestreadas en la segunda etapa tienen, en promedio, muchos más amigos que las personas de la muestra original. Esto sugiere que, en promedio, tus amigos son más populares que tú.
5. Peleas de bar. Un estudio de las peleas de bares en los que alguien haya sido asesinado encontró que, en el 90% de los casos, la persona que comenzó la pelea era el fallecido.

## Teorema del límite central para el promedio:

La distribución del promedio de la muestra se puede aproximar bien a partir de un modelo normal:

$$ \bar{X} \sim \mbox{N}(\mu, SE ) $$


donde $SE$ representa el error estándar, que es definido como la desviación estándar de la distribución de muestreo, con muestras de tamaño $n$. Si $\sigma$ es desconocido, utilizamos $s$, que es la desviación estándar de la muestra (insesgada, o sea dividida por $n-1$). 
No profundizaremos en la prueba detallada de por qué $SE = \frac{\sigma}{\sqrt{n}}$, pero notemos que en la medida de $n$ aumenta, $SE$ disminuye. En la medida en que el tamaño de la muestra aumenta, podemos esperar que las muestran arrojen promedios de muestra más consistentes, dado que la variabilidad entre los promedios de la muestra sería menor. 


Se deben cumplir ciertas condiciones para que aplique el TLC:

Independiencia: Las observaciones muestreadas deben ser independientes. Esto es difícil de verificar, pero es más probable si:  
* Se utiliza un muestreo/asignación aleatorios y
* En caso de muestreos sin remplazo, n < 10% de la población.
* Tamaño de la muestra / "asimetría"(skewness): Ya sea que la distribución de la población sea normal o que tenga “skewness”, el tamaño de la muestra es grande. Mientras más “sesgada” sea la distribución de la población, necesitamos una muestra más grande para que aplique el CLT. 
* Para distribuciones moderadamente asimétricas, $n > 30$ es aceptable

Esto también es difícil de verificar para la población,  pero podemos chequearlo utilizando los datos de la muestra y asumir que la muestra refleja lo que es la población.  


**Importante:**

El teorema del límite central dice que la distribución de probabilidad de la suma o el promedio de una gran muestra aleatoria extraída con reemplazo será aproximadamente normal, **independientemente de la distribución de la población de la que se extraiga la muestra.**
